# Prompt Engineering

### Passing a Base64 encoded image

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-_C0-J9gAQA"

In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
!pip show openai

In [ ]:
import openai
print(openai.__version__)

In [ ]:
import base64
from openai import OpenAI

client = OpenAI()

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "lion.jpg"

# Getting the Base64 string
base64_image = encode_image(image_path)


response = openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    max_tokens=300,
)

print(response.choices[0].message.content)

clarification on "Image input requirements"

clarification on "Limitations"

# Extend the model with tools 

In [ ]:
# !pip install openai==0.28

In [ ]:
import openai

openai.api_key = 'sk-pD-VLSk9gAQA'

response = openai.ChatCompletion.create(
    model="gpt-4.1",  # or "gpt-4", "gpt-3.5-turbo"
    messages=[
        {"role": "user", "content": "What was a positive news story today?"}
    ]
)
print(response.choices[0].message["content"])

clarifications in "Using Tools"

## Use built-in tools

### Function calling

In [ ]:
# !pip install openai

In [1]:
# !pip install --upgrade openai

In [2]:
import openai
print(openai.__version__)

1.39.0


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-_U-D-VLSk9gAQA"

#### get weather

In [ ]:

import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")  

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Bogotá, Colombia"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

response = openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "user", "content": "What is the weather like in Paris today?"}
    ],
    tools=tools,
    tool_choice="auto"
)

print(response.choices[0].message)


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ynMouujfxjTwRwO2WzPrPeBd', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')], refusal=None, annotations=[])


#### send email

In [ ]:
import openai

# openai.api_key = "sk-proj-_C"

openai.api_key = os.getenv("OPENAI_API_KEY")

tools = [
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Send an email to a given recipient with a subject and message.",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {
                        "type": "string",
                        "description": "The recipient email address."
                    },
                    "subject": {
                        "type": "string",
                        "description": "Email subject line."
                    },
                    "body": {
                        "type": "string",
                        "description": "Body of the email message."
                    }
                },
                "required": ["to", "subject", "body"]
            }
        }
    }
]

response = openai.chat.completions.create(
    model="gpt-4-1106-preview", 
    messages=[
        {"role": "user", "content": "Can you send an email to ilan@example.com and katia@example.com saying hi?"}
    ],
    tools=tools,
    tool_choice="auto"
)

print(response.choices[0].message)


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8UURV1c0P4h2pufF96ic3NzB', function=Function(arguments='{"to": "ilan@example.com", "subject": "Saying Hi", "body": "Hi Ilan!"}', name='send_email'), type='function'), ChatCompletionMessageToolCall(id='call_skGBRzQYUbK37uKn2Ce0X745', function=Function(arguments='{"to": "katia@example.com", "subject": "Saying Hi", "body": "Hi Katia!"}', name='send_email'), type='function')], refusal=None, annotations=[])


#### Search knowledge base

In [ ]:
#Search knowledge base

import openai


openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "user", "content": "What is the weather in Paris today?"}
    ],
    functions=[
        {
            "name": "get_weather",
            "description": "Get current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country, e.g. Paris, France",
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call="auto"
)

print(response)

ChatCompletion(id='chatcmpl-BxMRKHew8YIQeEMyGkudo5saYgHq9', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Paris, France"}', name='get_weather'), tool_calls=None, refusal=None, annotations=[]))], created=1753487310, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_51e1070cf2', usage=CompletionUsage(completion_tokens=16, prompt_tokens=65, total_tokens=81, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))


In [11]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']